# ffm_mxnetVersion0.1

## 首先测试mxnet是否具有所需要的能力

### 关于每次只使用部分的参数

In [1]:
import mxnet as mx
from mxnet import ndarray as nd
import mxnet.autograd as ag

In [2]:
weight = nd.array([1,2,3,4,5])

weight.attach_grad()

In [3]:

with ag.record():
    loss = nd.array([1,2,3])*weight[:3]

loss.backward()
print(weight.grad)


[ 1.  2.  3.  0.  0.]
<NDArray 5 @cpu(0)>


### 关于能够通过抽取的方式取出参数更新

In [4]:
with ag.record():
    tempdata  = weight[[3,4,3]]
    tempdata2 = weight[[2,1,3]]
    loss = nd.array([1,2,3])*tempdata*tempdata2
    
loss.backward()
print(weight.grad)


[  0.  10.   4.  27.   4.]
<NDArray 5 @cpu(0)>


In [25]:
import numpy as np
temp = np.array([1,2,3])
temp[1:2]

array([2])

### 关于三维数组的使用

In [38]:
nd.random_normal(shape=(2,2))


[[ 1.55807102 -0.54594457]
 [-2.35562968  0.54144025]]
<NDArray 2x2 @cpu(0)>

## 其次完成对于矩阵按照index进行存取的方法

In [44]:
index_v1 = ['a','b','c']
index_v2 = ['0','1','2','3','4','5']
array = nd.random_normal(shape = (3,5))

In [46]:
# 我希望做这样的一个东西：
# array[['a'.'a'],['2','2','4']]

#### 解决方法：operator.itemgetter

####  step 1:  build a dict to get a map from name to index

In [53]:
rowIndex = {'a':0,'b':1,'c':2}
colIndex = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5}

#### step 2: transfor the name to real index

In [55]:
from operator import itemgetter
rowneed = ['a','a']
colneed = ['2','2','4']
rowReal = itemgetter(*rowneed)(rowIndex)
colReal = itemgetter(*colneed)(colIndex)

In [56]:
print(rowReal,colReal)

(0, 0) (2, 2, 4)


### step 3:  get the value from dict , and change it 

In [57]:
array = nd.random_normal(shape = (3,5))

In [71]:
#use these mathod to get special row & cols
def selectM(list1, list2):
    row = np.array([[i] * len(list2) for i in list1])
    col = np.array(list2 * len(list1)).reshape(len(list1), len(list2))
    return row,col

In [73]:
row,col = selectM(rowReal,colReal)
print(row)
print(col)


[[0 0 0]
 [0 0 0]]
[[2 2 4]
 [2 2 4]]


In [76]:
array[row,col] = 1

In [77]:
array


[[-1.59187567 -1.10819471  1.         -0.91856343  1.        ]
 [-1.20808625  1.81402123 -1.52274299 -2.51524496 -1.35493255]
 [-0.95748401 -0.72485566  1.11196363 -0.47827262 -1.17394924]]
<NDArray 3x5 @cpu(0)>

In [ ]:
np.ones((4000,5)).dot(np.ones((5,))).shape

### make a function to use later

In [7]:
import numpy as np
rowIndex = {'a':0,'b':1,'c':2}
colIndex = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5}
rowneed = ['a','a']
colneed = ['2','2','4']
array = nd.random_normal(shape = (3,5))



def selectM(list1, list2):
    row = np.array([[i] * len(list2) for i in list1])
    col = np.array(list2 * len(list1)).reshape(len(list1), len(list2))
    return row,col

def askByIndex(rowneed,colneed,rowIndex,colIndex):
    from operator import itemgetter
    rowReal = itemgetter(*rowneed)(rowIndex)
    colReal = itemgetter(*colneed)(colIndex)
    row,col = selectM(rowReal,colReal)
    return row,col

row,col = askByIndex(rowneed,colneed,rowIndex,colIndex)
array[row,col]


[[-1.57344317 -1.57344317  0.29670075]
 [-1.57344317 -1.57344317  0.29670075]]
<NDArray 2x3 @cpu(0)>

### make it more easier

In [33]:
# make it more easier:

class DataManageMent(object):
    def __init__(self,rowIndex,colIndex):
        self._rowIndex = rowIndex
        self._colIndex = colIndex
    
    def selectM(self,list1, list2):
        row = np.array([[i] * len(list2) for i in list1])
        col = np.array(list2 * len(list1)).reshape(len(list1), len(list2))
        return row,col

    def askByIndex(self,rowneed,colneed):
        from operator import itemgetter
        rowReal = itemgetter(*rowneed)(self._rowIndex)
        colReal = itemgetter(*colneed)(self._colIndex)
        row,col = selectM(rowReal,colReal)
        return row,col
    def getByIndex(self,array,rowneed,colneed):
        row , col = self.askByIndex(rowneed,colneed)
        return array[row,col]


In [34]:
import numpy as np

# build a datamanager:
rowIndex = {'a':0,'b':1,'c':2}
colIndex = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5}
datamanager  = DataManageMent(rowIndex,colIndex)


rowneed = ['a','a']
colneed = ['2','2','4']
array = nd.random_normal(shape = (3,5))


datamanager.getByIndex(array,rowneed,colneed)



[[ 0.50805062  0.50805062  1.68393028]
 [ 0.50805062  0.50805062  1.68393028]]
<NDArray 2x3 @cpu(0)>

### mxnet version

In [27]:
import pandas as pd
import numpy as np
import os


In [28]:
trainSet = pd.read_csv("train.tiny-Copy1.csv")
trainSet.head()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000000,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,10000001,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,10000002,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,10000003,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,10000004,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


### 暂时跳过缺失值处理

In [29]:
trainSet = trainSet.dropna(how = 'any')
trainSet.head(4)

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
35,10000035,1,0.0,127,1.0,3.0,1683.0,19.0,26.0,17.0,...,8efede7f,bd17c3da,f6a3e43b,a458ea53,35cd95c9,ad3062eb,c7dc6720,3fdb382b,010f6491,49d68486
256,10000256,0,4.0,0,128.0,42.0,7.0,44.0,4.0,44.0,...,27c07bd6,bc5a0ff7,083e89d9,a458ea53,ac3aac58,c9d4222a,32c7478e,8e183052,001f3601,b4aa4b3d
328,10000328,0,3.0,0,27.0,26.0,25.0,37.0,14.0,22.0,...,e5ba7672,456d734d,4764bf77,a458ea53,52fa6318,ad3062eb,3a171ecb,3e90e1b2,724b04da,26c64313
338,10000338,0,0.0,1,2.0,14.0,2109.0,43.0,7.0,18.0,...,3486227d,df4fffb7,21ddcdc9,b1252a9d,53fbce39,c9d4222a,32c7478e,57ef7a21,010f6491,779ff446


### 取出局部资源来调试算法
- 注意：实际中应当把训练集和测试集统一进行编号

In [33]:
trainSet = trainSet.loc[:,['Label','I1','I2','C17','C18','C19']].head()
trainSet.head()

,Label,I1,I2,C17,C18,C19
35,1,0.0,127,8efede7f,bd17c3da,f6a3e43b
256,0,4.0,0,27c07bd6,bc5a0ff7,083e89d9
328,0,3.0,0,e5ba7672,456d734d,4764bf77
338,0,0.0,1,3486227d,df4fffb7,21ddcdc9
643,0,1.0,0,27c07bd6,a78bd508,21ddcdc9


### change type so it can be done

In [34]:
target = trainSet.Label
trainSet = trainSet.drop( 'Label',axis=1)
categories = ['C17','C18','C19']
continues  = ['I1','I2']

# attentsion : 这样的apply是一个逐列的处理
trainSet.loc[:,categories] = trainSet.loc[:,categories].apply(lambda x : x.astype('category'))
trainSet.loc[:,continues] = trainSet.loc[:,continues].apply(lambda x : x.astype('float32'))


### get dummies

In [124]:
xdata_trnsf = pd.get_dummies(trainSet,prefix=categories,prefix_sep = ":",sparse = True)
xdata_trnsf.reset_index(inplace=True,drop=True)

In [125]:
# xdata_trnsf = pd.get_dummies(trainSet,prefix=categories,prefix_sep = ":",sparse = True)
xdata_trnsf

,I1,I2,C17:27c07bd6,C17:3486227d,C17:8efede7f,C17:e5ba7672,C18:456d734d,C18:a78bd508,C18:bc5a0ff7,C18:bd17c3da,C18:df4fffb7,C19:083e89d9,C19:21ddcdc9,C19:4764bf77,C19:f6a3e43b
0,0.0,127.0,0,0,1,0,0,0,0,1,0,0,0,0,1
1,4.0,0.0,1,0,0,0,0,0,1,0,0,1,0,0,0
2,3.0,0.0,0,0,0,1,1,0,0,0,0,0,0,1,0
3,0.0,1.0,0,1,0,0,0,0,0,0,1,0,1,0,0
4,1.0,0.0,1,0,0,0,0,1,0,0,0,0,1,0,0


In [126]:
# 拿到一级label和二级label,重构索引
import re
label_v1 = [ label.split(':')[0] for label in xdata_trnsf.columns.tolist()]
label_v2 = xdata_trnsf.columns.tolist()
index = pd.MultiIndex.from_arrays([label_v1,label_v2],names = ['field_id','index_id'])
xdata_trnsf.columns = index
xdata_trnsf.head()

field_id   I1     I2          C17                                         \
index_id   I1     I2 C17:27c07bd6 C17:3486227d C17:8efede7f C17:e5ba7672   
0         0.0  127.0            0            0            1            0   
1         4.0    0.0            1            0            0            0   
2         3.0    0.0            0            0            0            1   
3         0.0    1.0            0            1            0            0   
4         1.0    0.0            1            0            0            0   

field_id          C18                                                      \
index_id C18:456d734d C18:a78bd508 C18:bc5a0ff7 C18:bd17c3da C18:df4fffb7   
0                   0            0            0            1            0   
1                   0            0            1            0            0   
2                   1            0            0            0            0   
3                   0            0            0            0            1   
4                   0            1            0            0            0   

field_id          C19                                         
index_id C19:083e89d9 C19:21ddcdc9 C19:4764bf77 C19:f6a3e43b  
0                   0            0            0            1  
1                   1            0            0            0  
2                   0            0            1            0  
3                   0            1            0            0  
4                   0            1            0            0

### 制作字典，存储从名字到index 的映射

In [127]:
# define a function to print dict:
def printDict(d):
    import json
    print(json.dumps(d, indent = 4))

In [134]:
def createIndex(label):
    from collections import OrderedDict
    from collections import Counter
    #print(label)
    orderCol = list(OrderedDict.fromkeys(label))
    #print(orderCol)
    label_dict = dict(zip(orderCol,range(len(orderCol))))
    return label_dict

In [135]:
# encode:
label_v1_dict = createIndex(label_v1)
printDict(label_v1_dict)
label_v2_dict = createIndex(label_v2)
printDict(label_v2_dict)

{
    "I1": 0,
    "I2": 1,
    "C17": 2,
    "C18": 3,
    "C19": 4
}
{
    "I1": 0,
    "I2": 1,
    "C17:27c07bd6": 2,
    "C17:3486227d": 3,
    "C17:8efede7f": 4,
    "C17:e5ba7672": 5,
    "C18:456d734d": 6,
    "C18:a78bd508": 7,
    "C18:bc5a0ff7": 8,
    "C18:bd17c3da": 9,
    "C18:df4fffb7": 10,
    "C19:083e89d9": 11,
    "C19:21ddcdc9": 12,
    "C19:4764bf77": 13,
    "C19:f6a3e43b": 14
}


### 初始化模型参数

In [136]:
# tunning parameters
numOfField = label_v1_dict.__len__()
numOfIndex = label_v2_dict.__len__()
numOfK     = 1

In [137]:
import mxnet.ndarray as nd
import mxnet.autograd as ag
Wmatrix = nd.random_normal(shape=(numOfK,numOfField,numOfIndex))
Wmatrix.attach_grad()
print(Wmatrix)
print(Wmatrix.grad)


[[[-0.12628114 -2.13627362 -0.93159074  0.66304696 -2.01707149 -1.61501789
   -0.35159519 -0.35048652 -2.34388471  0.49228418 -0.83288866 -0.06915966
   -0.25536978 -0.76541281  0.36886221]
  [-0.51822037 -0.35396403 -0.9016903  -0.74716997 -1.41808152 -0.75003189
   -0.94927508 -0.31106019  0.63803208  0.26596701 -0.80320734 -0.33746058
    0.51752234 -0.15570128 -0.43022308]
  [ 0.59488088 -0.18376918 -1.10045826  1.53674662 -1.31767082  0.21463518
    0.05731938 -2.74431562  0.68399608  0.02551857  0.21768956  0.78508347
   -0.47509891  0.40476051 -0.60806912]
  [-1.52903938 -1.08527052 -0.82657975 -0.98754108  0.13191865  1.23952746
    1.86757302  0.87418163 -0.25921452  0.74323058 -0.28645536  1.03354466
   -0.66191357 -0.73003006 -0.69364911]
  [-1.30935442 -0.83390754 -1.4784236   0.50401926 -0.03488216 -1.28532219
   -0.26264143 -0.05429081 -0.47735032 -0.29853135 -0.65074015  0.95971918
    0.09444837  1.44644213  0.4567737 ]]]
<NDArray 1x5x15 @cpu(0)>

[[[ 0.  0.  0.  0.  0

### SGD

#### 获取第一个人的信息

In [186]:
from IPython.display import display_html
from scipy.sparse import  coo_matrix,csr_matrix,lil_matrix
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

record = xdata_trnsf.loc[0,:].to_frame().to_sparse()
record

0
field_id index_id           
I1       I1              0.0
I2       I2            127.0
C17      C17:27c07bd6    0.0
         C17:3486227d    0.0
         C17:8efede7f    1.0
         C17:e5ba7672    0.0
C18      C18:456d734d    0.0
         C18:a78bd508    0.0
         C18:bc5a0ff7    0.0
         C18:bd17c3da    1.0
         C18:df4fffb7    0.0
C19      C19:083e89d9    0.0
         C19:21ddcdc9    0.0
         C19:4764bf77    0.0
         C19:f6a3e43b    1.0


#### 生成交互矩阵

In [184]:
# 先看看交互矩阵长什么样
record = xdata_trnsf.loc[0,:].to_frame().to_sparse()
record
record_matrix = record.dot(record.T)
upper = np.triu(record_matrix.values)
upper[range(upper.shape[0]),range(upper.shape[0]) ]= 0
record_matrix = pd.DataFrame(upper  ,index = record_matrix.index , columns = record_matrix.columns)
record_matrix

field_id                I1   I2          C17                            \
index_id                I1   I2 C17:27c07bd6 C17:3486227d C17:8efede7f   
field_id index_id                                                        
I1       I1            0.0  0.0          0.0          0.0          0.0   
I2       I2            0.0  0.0          0.0          0.0        127.0   
C17      C17:27c07bd6  0.0  0.0          0.0          0.0          0.0   
         C17:3486227d  0.0  0.0          0.0          0.0          0.0   
         C17:8efede7f  0.0  0.0          0.0          0.0          0.0   
         C17:e5ba7672  0.0  0.0          0.0          0.0          0.0   
C18      C18:456d734d  0.0  0.0          0.0          0.0          0.0   
         C18:a78bd508  0.0  0.0          0.0          0.0          0.0   
         C18:bc5a0ff7  0.0  0.0          0.0          0.0          0.0   
         C18:bd17c3da  0.0  0.0          0.0          0.0          0.0   
         C18:df4fffb7  0.0  0.0          0.0          0.0          0.0   
C19      C19:083e89d9  0.0  0.0          0.0          0.0          0.0   
         C19:21ddcdc9  0.0  0.0          0.0          0.0          0.0   
         C19:4764bf77  0.0  0.0          0.0          0.0          0.0   
         C19:f6a3e43b  0.0  0.0          0.0          0.0          0.0   

field_id                                    C18                            \
index_id              C17:e5ba7672 C18:456d734d C18:a78bd508 C18:bc5a0ff7   
field_id index_id                                                           
I1       I1                    0.0          0.0          0.0          0.0   
I2       I2                    0.0          0.0          0.0          0.0   
C17      C17:27c07bd6          0.0          0.0          0.0          0.0   
         C17:3486227d          0.0          0.0          0.0          0.0   
         C17:8efede7f          0.0          0.0          0.0          0.0   
         C17:e5ba7672          0.0          0.0          0.0          0.0   
C18      C18:456d734d          0.0          0.0          0.0          0.0   
         C18:a78bd508          0.0          0.0          0.0          0.0   
         C18:bc5a0ff7          0.0          0.0          0.0          0.0   
         C18:bd17c3da          0.0          0.0          0.0          0.0   
         C18:df4fffb7          0.0          0.0          0.0          0.0   
C19      C19:083e89d9          0.0          0.0          0.0          0.0   
         C19:21ddcdc9          0.0          0.0          0.0          0.0   
         C19:4764bf77          0.0          0.0          0.0          0.0   
         C19:f6a3e43b          0.0          0.0          0.0          0.0   

field_id                                                 C19               \
index_id              C18:bd17c3da C18:df4fffb7 C19:083e89d9 C19:21ddcdc9   
field_id index_id                                                           
I1       I1                    0.0          0.0          0.0          0.0   
I2       I2                  127.0          0.0          0.0          0.0   
C17      C17:27c07bd6          0.0          0.0          0.0          0.0   
         C17:3486227d          0.0          0.0          0.0          0.0   
         C17:8efede7f          1.0          0.0          0.0          0.0   
         C17:e5ba7672          0.0          0.0          0.0          0.0   
C18      C18:456d734d          0.0          0.0          0.0          0.0   
         C18:a78bd508          0.0          0.0          0.0          0.0   
         C18:bc5a0ff7          0.0          0.0          0.0          0.0   
         C18:bd17c3da          0.0          0.0          0.0          0.0   
         C18:df4fffb7          0.0          0.0          0.0          0.0   
C19      C19:083e89d9          0.0          0.0          0.0          0.0   
         C19:21ddcdc9          0.0          0.0          0.0          0.0   
         C19:4764bf77          0.0          0.0 

In [190]:
# 生成交互矩阵
record = xdata_trnsf.iloc[0,:].to_frame().to_sparse()
record = csr_matrix(record.values)
record.toarray()

from scipy.sparse import triu
record_matrix = record.dot(record.T)
record_matrix = triu(record_matrix,format='lil')
record_diag = range(record_matrix.shape[0])
record_matrix[record_diag,record_diag ]= 0
record_matrix.tocoo().to_frame()

AttributeError: to_frame not found